In [2]:
# Activate Spark in our Colab notebook.
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [2,351 kB]
Get:14 http://sec

In [3]:
# Get postgresql package
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-01-18 06:16:17--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.26MB/s    in 0.7s    

2023-01-18 06:16:19 (1.26 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
# Import Spark and create a SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

# Extract the Amazon Data into Spark DataFrame

In [5]:
# Read in the data from an S3 Bucket
from pyspark import SparkFiles
url="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz" 

spark.sparkContext.addFile(url)
review_data_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True, timestampFormat="yyyy-mm-dd")

# Show DataFrame
review_data_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok|2015-01-31 00:08:00|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...| 

In [6]:
# Get the number of rows in the DataFrame.
print((review_data_df.count(), len(review_data_df.columns)))

(1801974, 15)


In [10]:
# Drop null values, by default is any which means will drop any rows which has a NA
cleaned_df = review_data_df.dropna(how='any')
# Drop any duplicate
cleaned_df = review_data_df.dropDuplicates() #cleaned_df.dropDuplicates()
print((cleaned_df.count(), len(cleaned_df.columns)))

(1801974, 15)


# Transform the Data

## Create the "review_id_table".

In [11]:
from pyspark.sql.functions import to_date
# Create the "review_id_df" DataFrame with the appropriate columns and data types.
review_id_df = cleaned_df.select(['review_id','customer_id','product_id','product_parent','review_date'])
review_id_df.show()

+--------------+-----------+----------+--------------+-------------------+
|     review_id|customer_id|product_id|product_parent|        review_date|
+--------------+-----------+----------+--------------+-------------------+
|R100D8YH7X7B3X|   37117341|B003GTSHY8|     857227973|2013-01-04 00:05:00|
|R100MTYMDVRFUP|   14113755|B0032JRRXO|     746382055|2014-01-09 00:07:00|
|R1018FYJ1WDQW5|   51233024|B0007WK8LC|      40553661|2013-01-04 00:01:00|
|R101CTJD0DGOY9|   32961533|B00006JN3G|     823925280|2008-01-19 00:05:00|
|R102FGD0WQ78MJ|     114958|B00UUQD9L6|     648963813|2015-01-18 00:08:00|
|R1038GUTY6PXA7|   29057034|B004WP02FM|     909899958|2013-01-21 00:07:00|
|R104A69PGL5Z1Y|   24899002|B0000DCW2V|      28618136|2014-01-28 00:08:00|
|R106BX1CHHOW4C|   35353346|B004DMNT2W|      34844312|2011-01-07 00:03:00|
|R106LZBR2ACVAX|    2874307|B000GUKTDM|     919273572|2013-01-06 00:09:00|
|R107MAJ145QM3W|    4421625|B00F0S0DK4|     388937146|2015-01-14 00:02:00|
|R107W8FT9A92VL|     2687

## Create the "products" Table

In [12]:
# Create the "products_df" DataFrame that drops the duplicates in the "product_id" and "product_title columns. 
products_df = cleaned_df.select(['product_id','product_title'])
products_df = products_df.distinct()
products_df.count()

168675

## Create the "customers" Table

In [13]:
# Create the "customers_df" DataFrame that groups the data on the "customer_id" by the number of times a customer reviewed a product. 
customers_df = cleaned_df.groupby('customer_id').agg({'customer_id':"count"})
customers_df = customers_df.orderBy('count(customer_id)',ascending = False)
customers_df = customers_df.withColumnRenamed('count(customer_id)','customer_count')
customers_df = customers_df.dropna(how='any')
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   31588426|           285|
|   50820654|           191|
|   52764559|           171|
|   44777060|           148|
|   52340667|           146|
|   45664110|           145|
|    9115336|           140|
|   53090839|           130|
|   52859210|           129|
|   24550970|           126|
|   45371561|           126|
|   27140716|           126|
|   40109303|           123|
|   51865210|           120|
|   38681283|           114|
|   22401847|           112|
|   52770861|           107|
|   48640632|           104|
|   52765756|           103|
|   50906184|           102|
+-----------+--------------+
only showing top 20 rows



## Create the "vine_table".

In [14]:
# Create the "vine_df" DataFrame that has the "review_id", "star_rating", "helpful_votes", "total_votes", and "vine" columns. 
vine_df = cleaned_df.select(['review_id','star_rating','helpful_votes','total_votes','vine'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R100D8YH7X7B3X|          3|            0|          0|   N|
|R100MTYMDVRFUP|          4|            0|          0|   N|
|R1018FYJ1WDQW5|          5|            0|          0|   N|
|R101CTJD0DGOY9|          4|            0|          2|   N|
|R102FGD0WQ78MJ|          5|            0|          0|   N|
|R1038GUTY6PXA7|          4|            4|          6|   N|
|R104A69PGL5Z1Y|          5|            0|          0|   N|
|R106BX1CHHOW4C|          4|            1|          1|   Y|
|R106LZBR2ACVAX|          5|            1|          1|   N|
|R107MAJ145QM3W|          5|            0|          0|   N|
|R107W8FT9A92VL|          4|            0|          0|   N|
|R108NUYT98TB93|          5|            0|          0|   N|
|R1090TKVUN08BT|          1|            0|          0|   N|
|R109BET36H73P1|          5|            

# Load

In [19]:
mode = "append"
jdbc_url="jdbc:postgresql://big-data-2.clo5tanjlfil.ap-southeast-2.rds.amazonaws.com:5432/amazon_review_part1_data2"
config = {"user":"postgres", "password": "qwertyui", "driver":"org.postgresql.Driver"}

In [21]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [20]:
# Write products_df to table in RDS
products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [22]:
# Write customers_df to table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [23]:
# Write vine_df to table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)